In [ ]:
import boto3
import os
from Network_Security.logging.logger import logging

class S3Client:
    s3_client = None
    s3_resource = None

    def __init__(self, region='us-east-1'):
        _access_key = os.getenv('AWS_ACCESS_KEY')
        _security_key = os.getenv('AWS_SECURITY_KEY')

        if _access_key is None:
            logging.info("Environment variable 'AWS_ACCESS_KEY' is not available")
        if _security_key is None:
            logging.info("Environment variable 'AWS_SECURITY_KEY' is not available")

        if _access_key is None or _security_key is None:
            raise ValueError("Missing AWS credentials in environment variables")

        if S3Client.s3_client is None or S3Client.s3_resource is None:
            S3Client.s3_resource = boto3.resource(
                's3',
                aws_access_key_id=_access_key,
                aws_secret_access_key=_security_key,
                region_name=region
            )
            S3Client.s3_client = boto3.client(
                's3',
                aws_access_key_id=_access_key,
                aws_secret_access_key=_security_key,
                region_name=region
            )

        # Assign to instance
        self.s3_client = S3Client.s3_client
        self.s3_resource = S3Client.s3_resource 


In [ ]:
class Network_service:
    def __init__(self,bucket_name,model_path):
        self.bucket_name = bucket_name 
        self.model_name = model_path 
        self.s3 = S3Storage_service()
        
    def is_present_model(self,modelpath):
        return self.s3.s3_key_path_avilable(bucket_name = self.bucket_name,
                                            se_key = modelpath)



class S3Storage_service:
    def __init__(self):
        S3_lient = S3Client()
        self.bucket_name = S3_lient.bucket_name
        self.model_path = S3_lient.model_path

    def s3_key_path_avilable(self,bucket_name,s3_key):
        bucket = self.bucket_name.Bucket(bucket_name)
        file_obj = [file_obj for file_obj in bucket.objects.filter(prefix=s3_key)]
        if len(file_obj)>0:
            return file_obj 
        return None


In [ ]:
# .env 
AWS_ACCESS_KEY = ''
AWS_SECRITE_KEY = ''

# constant 
AWS_ACCESS_KEY = 'AWS_ACCESS_KEY'
AWS_SECRITE_KEY = 'AWS_SECRITE_KEY' 
REGION_KEY = 'us-east-1'

MODEL_EVALUTION_CHANGED_THRESHOLD:float = 0.7
MODEL_TRAINER_TRAINED_MODEL_NAME = 'model.pkl'
MODEL_BUCKET_NAME = ''

# config 
from dataclasses import dataclass 
@dataclass 
class ModelEvalutionConfig:
    bucket_name:str = MODEL_BUCKET_NAME 
    s3_model_path:str = MODEL_TRAINER_TRAINED_MODEL_NAME
    changed_model_score:float =  MODEL_EVALUTION_CHANGED_THRESHOLD

@dataclass 
class ModelEvalutionResponse:
    is_model_accept:bool
    difference:float 
    train_model_f1_score:float 
    s3_model_f1_score:float 

@dataclass 
class ModelEvalationArtifact:
    is_model_accepted:bool 
    changed_accuracy:float
    train_model_path:str 
    s3_model_path:str 

    

from Network_Security.entity.artifact import (Data_Ingestion_Artifact,Model_Trainer_Artifact)
from Network_Security.constant import TARGET_COLUMN
from sklearn.metrics import f1_score
import pandas as pd

class ModelEvulation:
    def __init__(self,data_ingestion_artiface=Data_Ingestion_Artifact,
                 model_train_artifact=Model_Trainer_Artifact,
                 model_evalution_config=ModelEvalutionConfig):
        self.data_ingestion_artiface = data_ingestion_artiface
        self.model_train_artifact = model_train_artifact 
        self.model_evalution_config = model_evalution_config

    def get_model_present(self):
        bucket_name = self.model_evalution_config.bucket_name 
        model_path = self.model_evalution_config.s3_model_path 

        network_model = Network_service(bucket_name=bucket_name,
                                        model_path=model_path)
        if network_model.is_present_model(model_path):
            return network_model 
        return None 
    
    def evaluate_model(self):
        test_df = pd.read_csv(self.data_ingestion_artiface.test_file_path)
        x = test_df.drop([TARGET_COLUMN],axis=1)
        y = test_df([TARGET_COLUMN])
        y = y.repalce(-1,0)
        best_model = self.get_model_present()
        train_model_score = self.model_train_artifact.metrics.f1_score
        best_model_score = None
        if best_model is not None:
            pred = best_model.predict(x)
            best_model_score = f1_score(y,pred)
        temp_model_score = 0 if best_model_score is None else best_model_score
        model_evalution_response = ModelEvalutionResponse(is_model_accept=train_model_score>temp_model_score,
                                                          change_accuracy=train_model_score - temp_model_score,
                                                          train_model_f1_score=train_model_score,
                                                          s3_model_f1_score=best_model_score)
        return model_evalution_response
    
    def init_evalute_model(self):
        model_evalution_response = self.evaluate_model()
        s3_model = self.model_evalution_config.s3_model_path 
        model_evalution_artifact = ModelEvalationArtifact(is_model_accepted=model_evalution_response.is_model_accept,
                                                          difference=model_evalution_response.change_accuracy,
                                                          train_model_path=self.model_train_artifact.model_pkl,
                                                          s3_model_path = s3_model)
        return model_evalution_artifact 

def start_model_evalution(self,data_ingestion_artiface : Data_Ingestion_Artifact,
                 model_train_artifact : Model_Trainer_Artifact):
    model_evalute = ModelEvulation(model_evalution_config=self.model_evalution_config,
                                   data_ingestion_artiface=data_ingestion_artiface,
                                   model_train_artifact=model_train_artifact)
    model_evalution_artifact = model_evalute.init_evalute_model()
    return model_evalution_artifact




